In [2]:
import requests
import pandas as pd
import sqlite3
from datetime import datetime

### OMDB API for Movies

In [27]:
# etl_process.ipynb



# API Key and Base URL
API_KEY = "2c6d4f8"  # Replace with your API key
BASE_URL = "http://www.omdbapi.com/"

# Function to Extract Data
def extract_movie_data(movie_titles):
    movie_data = []
    for title in movie_titles:
        response = requests.get(BASE_URL, params={"t": title, "apikey": API_KEY})
        if response.status_code == 200:
            data = response.json()
            if data.get("Response") == "True":
                movie_data.append(data)
    return movie_data

# Function to Transform Data
def transform_movie_data(movie_data):
    transformed_data = []
    for movie in movie_data:
        transformed_data.append({
            "Title": movie.get("Title"),
            "Genre": movie.get("Genre"),
            "Director": movie.get("Director"),
            "IMDB_Rating": float(movie.get("imdbRating", 0)),
            "Year": int(movie.get("Year", 0)),
        })
    return pd.DataFrame(transformed_data)

# Function to Load Data into SQLite
def load_data_to_sqlite(df, db_name="movie_ratings.db"):
    conn = sqlite3.connect(db_name)
    df.to_sql("ratings", conn, if_exists="replace", index=False)
    conn.close()

# List of Movie Titles to Process
movie_titles = ["Inception", "The Dark Knight", "Interstellar", "The Matrix", "Gladiator"]

# ETL Process
if __name__ == "__main__":
    print("Starting ETL Process...")
    
    # Extract
    raw_data = extract_movie_data(movie_titles)
    print("Data Extracted Successfully!")

    # Transform
    transformed_data = transform_movie_data(raw_data)
    print("Data Transformed Successfully!")

    # Load
    load_data_to_sqlite(transformed_data)
    print("Data Loaded Successfully into SQLite!")


Starting ETL Process...
Data Extracted Successfully!
Data Transformed Successfully!
Data Loaded Successfully into SQLite!


### OpenWeatherMap Api for weather

In [12]:
# OpenWeatherMap API key and base URL
API_KEY = "118b63f0faa9cb1504b34a6b6810b30b"
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

# Cities to collect weather data for
cities = ["London", "New York", "Tokyo", "Sydney", "Mumbai"]

# Function to extract weather data
def extract_weather_data(cities):
    weather_data = []
    for city in cities:
        response = requests.get(BASE_URL, params={"q": city, "appid": API_KEY, "units": "metric"})
        if response.status_code == 200:
            data = response.json()
            print(data)
            weather_data.append(data)
        else:
            print(f"Failed to fetch data for {city}. Status Code: {response.status_code}")
    return weather_data

# Transform and collect all weather data
def transform_weather_data(weather_data):
    transformed_data = []
    for city_data in weather_data:
        transformed_data.append({
            "City": city_data.get("name"),
            "Temperature (°C)": city_data.get("main", {}).get("temp"),
            "Humidity (%)": city_data.get("main", {}).get("humidity"),
            "Wind Speed (m/s)": city_data.get("wind", {}).get("speed"),
            "Weather": city_data.get("weather", [{}])[0].get("description"),
            "Timestamp": datetime.now()
        })
    return pd.DataFrame(transformed_data)

# Load data into SQLite
def load_weather_data_to_sqlite(df):
    conn = sqlite3.connect("weather_data.db")
    df.to_sql("weather", conn, if_exists="replace", index=False)
    conn.close()
    print("Weather data loaded into SQLite database.")

# ETL Process
if __name__ == "__main__":
    print("Starting ETL Process...")
    
    # Extract
    raw_data = extract_weather_data(cities)
    print("Data Extracted Successfully!")

    # Transform
    transformed_data = transform_weather_data(raw_data)
    print("Data Transformed Successfully!")
    print(transformed_data)

     # Load
    load_weather_data_to_sqlite(transformed_data)
    print("Data Loaded Successfully into SQLite!")


Starting ETL Process...
{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 8.15, 'feels_like': 4.66, 'temp_min': 7.27, 'temp_max': 8.97, 'pressure': 983, 'humidity': 79, 'sea_level': 983, 'grnd_level': 979}, 'visibility': 10000, 'wind': {'speed': 6.69, 'deg': 190, 'gust': 12.86}, 'clouds': {'all': 20}, 'dt': 1737995434, 'sys': {'type': 2, 'id': 2075535, 'country': 'GB', 'sunrise': 1737963958, 'sunset': 1737996020}, 'timezone': 0, 'id': 2643743, 'name': 'London', 'cod': 200}
{'coord': {'lon': -74.006, 'lat': 40.7143}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 2.52, 'feels_like': -1.95, 'temp_min': 1.57, 'temp_max': 3.48, 'pressure': 1016, 'humidity': 41, 'sea_level': 1016, 'grnd_level': 1014}, 'visibility': 10000, 'wind': {'speed': 5.36, 'deg': 322, 'gust': 6.71}, 'clouds': {'all': 0}, 'dt':

In [11]:
import requests
import pandas as pd
from datetime import datetime
import sqlite3

# OpenWeatherMap API key and base URL for forecast
API_KEY = "118b63f0faa9cb1504b34a6b6810b30b"
BASE_URL_FORECAST = "https://api.openweathermap.org/data/2.5/forecast"

# Cities to collect weather data for
cities = ["London", "New York", "Tokyo", "Sydney", "Mumbai"]

# Function to extract forecast data
def extract_forecast_data(cities):
    forecast_data = []
    for city in cities:
        response = requests.get(BASE_URL_FORECAST, params={"q": city, "appid": API_KEY, "units": "metric"})
        if response.status_code == 200:
            data = response.json()
            print(data)
            forecast_data.append(data)
        else:
            print(f"Failed to fetch data for {city}. Status code: {response.status_code}")
    return forecast_data

# Function to transform forecast data
def transform_forecast_data(forecast_data):
    transformed_data = []
    for city_data in forecast_data:
        city_name = city_data["city"]["name"]
        for forecast in city_data["list"]:
            transformed_data.append({
                "City": city_name,
                "DateTime": forecast["dt_txt"],
                "Temperature": forecast["main"]["temp"],
                "Humidity": forecast["main"]["humidity"],
                "Wind Speed": forecast["wind"]["speed"],
                "Weather": forecast["weather"][0]["description"]
            })
    return pd.DataFrame(transformed_data)

# Function to load data into SQLite
def load_forecast_data_to_sqlite(df):
    conn = sqlite3.connect("weather_data.db")
    df.to_sql("forecast", conn, if_exists="replace", index=False)
    conn.close()
    print("Forecast data loaded into SQLite database.")

# ETL Process
if __name__ == "__main__":
    print("Starting Forecast ETL Process...")
    
    # Extract
    raw_forecast_data = extract_forecast_data(cities)
    print("Forecast Data Extracted Successfully!")

    # Transform
    transformed_forecast_data = transform_forecast_data(raw_forecast_data)
    print("Forecast Data Transformed Successfully!")

    # Load
    load_forecast_data_to_sqlite(transformed_forecast_data)
    print("Forecast Data Loaded Successfully into SQLite!")


Starting Forecast ETL Process...
{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1738000800, 'main': {'temp': 7.9, 'feels_like': 4.02, 'temp_min': 7.49, 'temp_max': 7.9, 'pressure': 983, 'sea_level': 983, 'grnd_level': 979, 'humidity': 77, 'temp_kf': 0.41}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'clouds': {'all': 40}, 'wind': {'speed': 7.72, 'deg': 208, 'gust': 17.38}, 'visibility': 10000, 'pop': 0.74, 'rain': {'3h': 0.34}, 'sys': {'pod': 'n'}, 'dt_txt': '2025-01-27 18:00:00'}, {'dt': 1738011600, 'main': {'temp': 7.41, 'feels_like': 3.23, 'temp_min': 7.07, 'temp_max': 7.41, 'pressure': 983, 'sea_level': 983, 'grnd_level': 979, 'humidity': 74, 'temp_kf': 0.34}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'clouds': {'all': 42}, 'wind': {'speed': 8.28, 'deg': 201, 'gust': 16.39}, 'visibility': 10000, 'pop': 0.2, 'rain': {'3h': 0.14}, 'sys': {'pod': 'n'}, 'dt_txt': '2025-01-27 21:00:00'}, {'dt'

In [15]:
# Connect to SQLite Database
conn = sqlite3.connect("weather_data.db")

# Load Data from SQLite
query = "SELECT * FROM forecast;"
data = pd.read_sql(query, conn)
conn.close()

# Display the First Few Rows
print("First few rows of the data:")
display(data.head(15))

First few rows of the data:


,City,DateTime,Temperature,Humidity,Wind Speed,Weather
0,London,2025-01-27 18:00:00,7.90,77,7.72,light rain
1,London,2025-01-27 21:00:00,7.41,74,8.28,light rain
2,London,2025-01-28 00:00:00,7.01,86,8.89,light rain
3,London,2025-01-28 03:00:00,7.67,85,8.75,light rain
4,London,2025-01-28 06:00:00,8.12,85,7.13,light rain
5,London,2025-01-28 09:00:00,8.42,85,6.68,light rain
6,London,2025-01-28 12:00:00,9.52,79,4.65,light rain
7,London,2025-01-28 15:00:00,7.61,86,4.67,light rain
8,London,2025-01-28 18:00:00,7.15,85,8.02,light rain
9,London,2025-01-28 21:00:00,6.48,79,6.56,overcast clouds
